In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

Referenz:
- https://stackoverflow.com/questions/49823192/autoencoder-gridsearch-hyperparameter-tuning-keras
- https://colab.research.google.com/drive/1TXaQzsSj2q0E3Ni1uxFDXGpY1SCnu46v?usp=sharing#scrollTo=6TSdu3Uk7ASm

# Daten vorverarbeiten

In [2]:
df = pd.read_pickle("Daten/data.pkl")
df_ohne_id = df.drop(columns=["id"])
feature_names = df_ohne_id.columns
feature_cnt = df_ohne_id.shape[1]

random_state = 21

x_train, x_test, y_train, y_test = train_test_split(df_ohne_id, np.zeros(shape=df_ohne_id.shape[0]), test_size=0.2, random_state=random_state)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=random_state)

del y_train
del y_val
del y_test

scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# Modell definieren

In [9]:
class AutoencoderTuner(Model):
    def __init__(self, hp, feature_cnt):
        super(AutoencoderTuner, self).__init__()
        
        nr_units_layer_2_4 = hp.Int("nr_units_layer_2_4", min_value=4, max_value=14, step=1)
        # nr_units_layer_3 = hp.Int("nr_units_layer_3", min_value=1, max_value=4, step=1) # Mittlere Schicht zu variieren ergibt keinen Sinn, da es für den Tuner immer besser ist hier möglichst viele Knoten zu verwenden
        nr_units_layer_3 = 4
        
        self.encoder = keras.Sequential([
            layers.Dense(nr_units_layer_2_4, activation="relu", name="layer_2"),
            layers.Dense(nr_units_layer_3, activation="relu", name="layer_3_middle"),
        ], name="encoder")
        self.decoder = keras.Sequential([
            layers.Dense(nr_units_layer_2_4, activation="relu", name="layer_4"),
            layers.Dense(feature_cnt, activation="sigmoid", name="layer_5_output"),
        ], name="decoder")

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
def build_model(hp):
    model = AutoencoderTuner(hp, feature_cnt)
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss='mse')
    return model

# Hyperparametersuche starten

In [10]:
tuner = kt.Hyperband(
    build_model,
    objective="val_loss",
    max_epochs=20,
    directory="hyperparameteroptimierung",
    project_name="hyperparameteroptimierung",
    seed=random_state
)

tuner.search(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
    validation_data=(x_val_scaled, x_val_scaled)
)

INFO:tensorflow:Reloading Oracle from existing project hyperparameteroptimierung\hyperparameteroptimierung\oracle.json
INFO:tensorflow:Reloading Tuner from hyperparameteroptimierung\hyperparameteroptimierung\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [11]:
hparams = ["nr_units_layer_2_4", "learning_rate"]
best_hyperparams = tuner.get_best_hyperparameters()
for hps in hparams:
    print(f"{hps}: {best_hyperparams[0][hps]}")

nr_units_layer_2_4: 7
learning_rate: 0.01
